In [1]:
import pandas as pd
from glob import glob
from time import strftime, sleep
import numpy as np
from datetime import datetime
from pandas_datareader import data as pdr
from pandas.tseries.offsets import BDay
import yfinance as yf
yf.pdr_override()

In [2]:
# simple function to make headers nicer
def clean_header(df):
    df.columns = df.columns.str.strip().str.lower().str.replace('.', '').str.replace('(', '').str.replace(')', '').str.replace(' ', '_').str.replace('_/_', '/')

# timestamp for file names
def get_now():
    now = datetime.now().strftime('%Y-%m-%d_%Hh%Mm')
    return now

In [3]:
last_file = glob('../inputs/transactions_all/transactions*.xlsx')[-1] # path to file in the folder
print(last_file[-(len(last_file))+(last_file.rfind('/')+1):])
all_transactions = pd.read_excel(last_file)
all_transactions.date = pd.to_datetime(all_transactions.date, format='%d/%m/%Y')

transactions_all\transactions_finaldf_2021-05-11_12h19m_repo.xlsx


In [4]:
all_tickers = list(all_transactions['ticker'].unique())
# some tickers may have been delisted. need to blacklist them here
blacklist = ['VSLR', 'HTZ']
filt_tickers = [tick for tick in all_tickers if tick not in blacklist]
print('You traded {} different stocks'.format(len(all_tickers)))

You traded 41 different stocks


In [5]:
# all transactions without the delisted stocks
final_filtered = all_transactions[~all_transactions.ticker.isin(blacklist)]

## Collecting the price history for all tickers

In [6]:
ly = datetime.today().year-1
today = datetime.today()
start_sp = datetime(2019, 1, 1)
end_sp = today
start_stocks = datetime(2019, 1, 1)
end_stocks = today
start_ytd = datetime(ly, 12, 31) + BDay(1)

def get(tickers, startdate, enddate):
    def data(ticker):
        return (pdr.get_data_yahoo(ticker, start=startdate, end=enddate))
    datas = map(data, tickers)
    return(pd.concat(datas, keys=tickers, names=['ticker', 'date']))
               
all_data = get(filt_tickers, start_stocks, end_stocks)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [7]:
clean_header(all_data)

In [8]:
# saving all stock prices individually to the specified folder
for tick in filt_tickers:
    all_data.loc[tick].to_csv('../outputs/price_hist/{}_price_hist.csv'.format(tick))

In [9]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 22331 entries, (MMM, 2019-01-02 00:00:00) to (GILT, 2021-05-14 00:00:00)
Data columns (total 6 columns):
open         22331 non-null float64
high         22331 non-null float64
low          22331 non-null float64
close        22331 non-null float64
adj_close    22331 non-null float64
volume       22331 non-null int64
dtypes: float64(5), int64(1)
memory usage: 1.1+ MB


##### MEGA_DICT = dictionary with all the tickers as keys, and their ticker prices as DF
##### MEGA_DF = all the DF's from DICT, concatenated along the columns. Can use filter to select columns

In [10]:
MEGA_DICT = {}  # you have to create it first
min_date = '2020-01-01'  # optional
TX_COLUMNS = ['date','ticker', 'cashflow', 'cml_units', 'cml_cost', 'gain_loss']
tx_filt = all_transactions[TX_COLUMNS]  # keeping just the most relevant ones for now

for ticker in filt_tickers:
    prices_df = all_data[all_data.index.get_level_values('ticker').isin([ticker])].reset_index()
    ## Can add more columns like volume!
    PX_COLS = ['date', 'adj_close']
    prices_df = prices_df[prices_df.date >= min_date][PX_COLS].set_index(['date'])
    # Making sure we get sameday transactions
    tx_df = tx_filt[tx_filt.ticker==ticker].groupby('date').agg({'cashflow': 'sum',
                                                                 'cml_units': 'last',
                                                                 'cml_cost': 'last',
                                                                 'gain_loss': 'sum'})
    # Merging price history and transactions dataframe
    tx_and_prices = pd.merge(prices_df, tx_df, how='outer', left_index=True, right_index=True).fillna(0)
    # This is to fill the days that were not in our transaction dataframe
    tx_and_prices['cml_units'] = tx_and_prices['cml_units'].replace(to_replace=0, method='ffill')
    tx_and_prices['cml_cost'] = tx_and_prices['cml_cost'].replace(to_replace=0, method='ffill')
    tx_and_prices['gain_loss'] = tx_and_prices['gain_loss'].replace(to_replace=0, method='ffill')
    # Cumulative sum for the cashflow
    tx_and_prices['cashflow'] = tx_and_prices['cashflow'].cumsum()
    tx_and_prices['avg_price'] = (tx_and_prices['cml_cost']/tx_and_prices['cml_units'])
    tx_and_prices['mktvalue'] = (tx_and_prices['cml_units']*tx_and_prices['adj_close'])
    tx_and_prices = tx_and_prices.add_prefix(ticker+'_')
    # Once we're happy with the dataframe, add it to the dictionary
    MEGA_DICT[ticker] = tx_and_prices.round(3)

In [11]:
MEGA_DICT['RUN'].tail()

RUN_adj_close  RUN_cashflow  RUN_cml_units  RUN_cml_cost  \
date                                                                   
2021-05-10          40.80        201.08           10.0       336.386   
2021-05-11          42.94        201.08           10.0       336.386   
2021-05-12          39.63        201.08           10.0       336.386   
2021-05-13          39.45        201.08           10.0       336.386   
2021-05-14          41.42        201.08           10.0       336.386   

            RUN_gain_loss  RUN_avg_price  RUN_mktvalue  
date                                                    
2021-05-10        429.891         33.639         408.0  
2021-05-11        429.891         33.639         429.4  
2021-05-12        429.891         33.639         396.3  
2021-05-13        429.891         33.639         394.5  
2021-05-14        429.891         33.639         414.2

In [12]:
MEGA_DF = pd.concat(MEGA_DICT.values(), axis=1)
MEGA_DF.to_csv('../outputs/mega/MEGA_DF_{}.csv'.format(get_now()))  # optional
MEGA_DF.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 345 entries, 2020-01-02 to 2021-05-14
Columns: 273 entries, MMM_adj_close to GILT_mktvalue
dtypes: float64(273)
memory usage: 738.5 KB


In [13]:
last_file = glob('../outputs/mega/MEGA*.csv')[-1] # path to file in the folder
print(last_file[-(len(last_file))+(last_file.rfind('/')+1):])
MEGA_DF = pd.read_csv(last_file)

MEGA_DF['date'] = pd.to_datetime(MEGA_DF['date'])
MEGA_DF.set_index('date', inplace=True)

mega\MEGA_DF_2021-05-15_13h03m.csv


## Portfolio DF

In [14]:
portf_allvalues = MEGA_DF.filter(regex='mktvalue').fillna(0) #  getting just the market value of each ticker
portf_allvalues['portf_value'] = portf_allvalues.sum(axis=1) # summing all market values
portf_allvalues['portf_value']

date
2020-01-02        0.000
2020-01-03        0.000
2020-01-06        0.000
2020-01-07        0.000
2020-01-08      664.862
                ...    
2021-05-10    24491.288
2021-05-11    24775.617
2021-05-12    23923.571
2021-05-13    23845.160
2021-05-14    24721.358
Name: portf_value, Length: 345, dtype: float64

In [15]:
# For the S&P500 price return
sp500 = pdr.get_data_yahoo('^GSPC', start_stocks, end_sp)
clean_header(sp500)

[*********************100%***********************]  1 of 1 completed


In [16]:
#getting the pct change
portf_allvalues = portf_allvalues.join(sp500['adj_close'], how='inner')
portf_allvalues.rename(columns={'adj_close': 'sp500_mktvalue'}, inplace=True)
portf_allvalues['ptf_value_pctch'] = (portf_allvalues['portf_value'].pct_change()*100).round(2)
portf_allvalues['sp500_pctch'] = (portf_allvalues['sp500_mktvalue'].pct_change()*100).round(2)
portf_allvalues['ptf_value_diff'] = (portf_allvalues['portf_value'].diff()).round(2)
portf_allvalues['sp500_diff'] = (portf_allvalues['sp500_mktvalue'].diff()).round(2)

In [17]:
portf_allvalues.head()

MMM_mktvalue  TSLA_mktvalue  AAPL_mktvalue  RUN_mktvalue  \
2020-01-02         0.000           0.00            0.0           0.0   
2020-01-03         0.000           0.00            0.0           0.0   
2020-01-06         0.000           0.00            0.0           0.0   
2020-01-07         0.000           0.00            0.0           0.0   
2020-01-08       172.722         492.14            0.0           0.0   

            ZM_mktvalue  DIS_mktvalue  IBM_mktvalue  AMD_mktvalue  \
2020-01-02          0.0           0.0           0.0           0.0   
2020-01-03          0.0           0.0           0.0           0.0   
2020-01-06          0.0           0.0           0.0           0.0   
2020-01-07          0.0           0.0           0.0           0.0   
2020-01-08          0.0           0.0           0.0           0.0   

            FB_mktvalue  BA_mktvalue  ...  PLTR_mktvalue  PLL_mktvalue  \
2020-01-02          0.0          0.0  ...            0.0           0.0   
2020-01-03          0.0          0.0  ...            0.0           0.0   
2020-01-06          0.0          0.0  ...            0.0           0.0   
2020-01-07          0.0          0.0  ...            0.0           0.0   
2020-01-08          0.0          0.0  ...            0.0           0.0   

            TLRY_mktvalue  GILT_mktvalue  portf_value  sp500_mktvalue  \
2020-01-02            0.0            0.0        0.000     3257.850098   
2020-01-03            0.0            0.0        0.000     3234.850098   
2020-01-06            0.0            0.0        0.000     3246.280029   
2020-01-07            0.0            0.0        0.000     3237.179932   
2020-01-08            0.0            0.0      664.862     3253.050049   

            ptf_value_pctch  sp500_pctch  ptf_value_diff  sp500_diff  
2020-01-02              NaN          NaN             NaN         NaN  
2020-01-03              NaN        -0.71            0.00      -23.00  
2020-01-06              NaN         0.35            0.00       11.43  
2020-01-07              NaN        -0.28            0.00       -9.10  
2020-01-08              inf         0.49          664.86       15.87  

[5 rows x 45 columns]

In [18]:
# KPI's for portfolio
kpi_portfolio7d_abs = portf_allvalues.tail(7).ptf_value_diff.sum().round(2)
kpi_portfolio15d_abs = portf_allvalues.tail(15).ptf_value_diff.sum().round(2)
kpi_portfolio30d_abs = portf_allvalues.tail(30).ptf_value_diff.sum().round(2)
kpi_portfolio200d_abs = portf_allvalues.tail(200).ptf_value_diff.sum().round(2)
kpi_portfolio7d_pct = (kpi_portfolio7d_abs/portf_allvalues.tail(7).portf_value[0]).round(3)*100
kpi_portfolio15d_pct = (kpi_portfolio15d_abs/portf_allvalues.tail(15).portf_value[0]).round(3)*100
kpi_portfolio30d_pct = (kpi_portfolio30d_abs/portf_allvalues.tail(30).portf_value[0]).round(3)*100
kpi_portfolio200d_pct = (kpi_portfolio200d_abs/portf_allvalues.tail(200).portf_value[0]).round(3)*100

In [19]:
# KPI's for S&P500
kpi_sp500_7d_abs = portf_allvalues.tail(7).sp500_diff.sum().round(2)
kpi_sp500_15d_abs = portf_allvalues.tail(15).sp500_diff.sum().round(2)
kpi_sp500_30d_abs = portf_allvalues.tail(30).sp500_diff.sum().round(2)
kpi_sp500_200d_abs = portf_allvalues.tail(200).sp500_diff.sum().round(2)
kpi_sp500_7d_pct = (kpi_sp500_7d_abs/portf_allvalues.tail(7).sp500_mktvalue[0]).round(3)*100
kpi_sp500_15d_pct = (kpi_sp500_15d_abs/portf_allvalues.tail(15).sp500_mktvalue[0]).round(3)*100
kpi_sp500_30d_pct = (kpi_sp500_30d_abs/portf_allvalues.tail(30).sp500_mktvalue[0]).round(3)*100
kpi_sp500_200d_pct = (kpi_sp500_200d_abs/portf_allvalues.tail(200).sp500_mktvalue[0]).round(3)*100

In [20]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Output, Input
import plotly.express as px
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
import dash_table
from jupyter_dash import JupyterDash

In [21]:
initial_date = '2020-05-30'  # do not use anything earlier than your first trade
plotlydf_portfval = portf_allvalues[portf_allvalues.index > initial_date]
plotlydf_portfval = plotlydf_portfval[['portf_value', 'sp500_mktvalue', 'ptf_value_pctch',
                                     'sp500_pctch', 'ptf_value_diff', 'sp500_diff']].reset_index().round(2)
# calculating cumulative growth since initial date
plotlydf_portfval['ptf_growth'] = plotlydf_portfval.portf_value/plotlydf_portfval['portf_value'].iloc[0]
plotlydf_portfval['sp500_growth'] = plotlydf_portfval.sp500_mktvalue/plotlydf_portfval['sp500_mktvalue'].iloc[0]
plotlydf_portfval.rename(columns={'index': 'date'}, inplace=True)  # needed for later
plotlydf_portfval

date  portf_value  sp500_mktvalue  ptf_value_pctch  sp500_pctch  \
0   2020-06-01     11477.15         3055.73             7.92         0.38   
1   2020-06-02     11621.27         3080.82             1.26         0.82   
2   2020-06-03     12160.49         3122.87             4.64         1.36   
3   2020-06-04     11940.24         3112.35            -1.81        -0.34   
4   2020-06-05     12501.47         3193.93             4.70         2.62   
..         ...          ...             ...              ...          ...   
237 2021-05-10     24491.29         4188.43            -3.30        -1.04   
238 2021-05-11     24775.62         4152.10             1.16        -0.87   
239 2021-05-12     23923.57         4063.04            -3.44        -2.14   
240 2021-05-13     23845.16         4112.50            -0.33         1.22   
241 2021-05-14     24721.36         4173.85             3.67         1.49   

     ptf_value_diff  sp500_diff  ptf_growth  sp500_growth  
0            841.94       11.42    1.000000      1.000000  
1            144.12       25.09    1.012557      1.008211  
2            539.22       42.05    1.059539      1.021972  
3           -220.25      -10.52    1.040349      1.018529  
4            561.24       81.58    1.089249      1.045227  
..              ...         ...         ...           ...  
237         -835.75      -44.17    2.133917      1.370681  
238          284.33      -36.33    2.158691      1.358792  
239         -852.05      -89.06    2.084452      1.329646  
240          -78.41       49.46    2.077620      1.345832  
241          876.20       61.35    2.153963      1.365909  

[242 rows x 9 columns]

In [22]:
CHART_THEME = 'plotly_white'  # others include seaborn, ggplot2, plotly_dark

chart_ptfvalue = go.Figure()  # generating a figure that will be updated in the following lines
chart_ptfvalue.add_trace(go.Scatter(x=plotlydf_portfval.date, y=plotlydf_portfval.portf_value,
                    mode='lines',  # you can also use "lines+markers", or just "markers"
                    name='Global Value'))
chart_ptfvalue.layout.template = CHART_THEME
chart_ptfvalue.layout.height=500
chart_ptfvalue.update_layout(margin = dict(t=50, b=50, l=25, r=25))  # this will help you optimize the chart space
chart_ptfvalue.update_layout(
#     title='Global Portfolio Value (USD $)',
    xaxis_tickfont_size=12,
    yaxis=dict(
        title='Value: $ USD',
        titlefont_size=14,
        tickfont_size=12,
        ))
# chart_ptfvalue.update_xaxes(rangeslider_visible=False)
# chart_ptfvalue.update_layout(showlegend=False)
chart_ptfvalue.show()

In [23]:
import plotly.io as pio
list(pio.templates)  # doctest: +ELLIPSIS

['ggplot2',
 'seaborn',
 'simple_white',
 'plotly',
 'plotly_white',
 'plotly_dark',
 'presentation',
 'xgridoff',
 'ygridoff',
 'gridon',
 'none']

In [24]:
plotlydf_portfval

date  portf_value  sp500_mktvalue  ptf_value_pctch  sp500_pctch  \
0   2020-06-01     11477.15         3055.73             7.92         0.38   
1   2020-06-02     11621.27         3080.82             1.26         0.82   
2   2020-06-03     12160.49         3122.87             4.64         1.36   
3   2020-06-04     11940.24         3112.35            -1.81        -0.34   
4   2020-06-05     12501.47         3193.93             4.70         2.62   
..         ...          ...             ...              ...          ...   
237 2021-05-10     24491.29         4188.43            -3.30        -1.04   
238 2021-05-11     24775.62         4152.10             1.16        -0.87   
239 2021-05-12     23923.57         4063.04            -3.44        -2.14   
240 2021-05-13     23845.16         4112.50            -0.33         1.22   
241 2021-05-14     24721.36         4173.85             3.67         1.49   

     ptf_value_diff  sp500_diff  ptf_growth  sp500_growth  
0            841.94       11.42    1.000000      1.000000  
1            144.12       25.09    1.012557      1.008211  
2            539.22       42.05    1.059539      1.021972  
3           -220.25      -10.52    1.040349      1.018529  
4            561.24       81.58    1.089249      1.045227  
..              ...         ...         ...           ...  
237         -835.75      -44.17    2.133917      1.370681  
238          284.33      -36.33    2.158691      1.358792  
239         -852.05      -89.06    2.084452      1.329646  
240          -78.41       49.46    2.077620      1.345832  
241          876.20       61.35    2.153963      1.365909  

[242 rows x 9 columns]

In [25]:
fig2 = go.Figure(data=[
    go.Bar(name='Portfolio', x=plotlydf_portfval['date'], y=plotlydf_portfval['ptf_value_pctch']),
    go.Bar(name='SP500', x=plotlydf_portfval['date'], y=plotlydf_portfval['sp500_pctch'])
])
# Change the bar mode
fig2.update_layout(barmode='group')
fig2.layout.template = CHART_THEME
fig2.layout.height=300
fig2.update_layout(margin = dict(t=50, b=50, l=25, r=25))
fig2.update_layout(
#     title='% variation - Portfolio vs SP500',
    xaxis_tickfont_size=12,
    yaxis=dict(
        title='% change',
        titlefont_size=14,
        tickfont_size=12,
        ))
fig2.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="right",
    x=0.99))

fig2.show()

In [26]:
df = plotlydf_portfval[['date', 'ptf_growth', 'sp500_growth']].copy().round(3)
df['month'] = df.date.dt.month_name()  # date column should be formatted as datetime
df['weekday'] = df.date.dt.day_name()  # could be interesting to analyze weekday returns later
df['year'] = df.date.dt.year
df['weeknumber'] = df.date.dt.week    # could be interesting to try instead of timeperiod
df['timeperiod'] = df.year.astype(str) + ' - ' + df.date.dt.month.astype(str).str.zfill(2)
df.head(5)

date  ptf_growth  sp500_growth month    weekday  year  weeknumber  \
0 2020-06-01       1.000         1.000  June     Monday  2020          23   
1 2020-06-02       1.013         1.008  June    Tuesday  2020          23   
2 2020-06-03       1.060         1.022  June  Wednesday  2020          23   
3 2020-06-04       1.040         1.019  June   Thursday  2020          23   
4 2020-06-05       1.089         1.045  June     Friday  2020          23   

  timeperiod  
0  2020 - 06  
1  2020 - 06  
2  2020 - 06  
3  2020 - 06  
4  2020 - 06

In [27]:
# getting the percentage change for each period. the first period will be NaN
sp = df.reset_index().groupby('timeperiod').last()['sp500_growth'].pct_change()*100
ptf = df.reset_index().groupby('timeperiod').last()['ptf_growth'].pct_change()*100
plotlydf_growth_compare = pd.merge(ptf, sp, on='timeperiod').reset_index().round(3)
plotlydf_growth_compare.head()

timeperiod  ptf_growth  sp500_growth
0  2020 - 06         NaN           NaN
1  2020 - 07       9.502         5.419
2  2020 - 08      32.290         7.009
3  2020 - 09       9.570        -3.843
4  2020 - 10      -5.054        -2.816

In [28]:
fig_growth2 = go.Figure()
fig_growth2.layout.template = CHART_THEME
fig_growth2.add_trace(go.Bar(
    x=plotlydf_growth_compare.timeperiod,
    y=plotlydf_growth_compare.ptf_growth.round(2),
    name='Portfolio'
))
fig_growth2.add_trace(go.Bar(
    x=plotlydf_growth_compare.timeperiod,
    y=plotlydf_growth_compare.sp500_growth.round(2),
    name='S&P 500',
))
fig_growth2.update_layout(barmode='group')
fig_growth2.layout.height=300
fig_growth2.update_layout(margin = dict(t=50, b=50, l=25, r=25))
fig_growth2.update_layout(
    xaxis_tickfont_size=12,
    yaxis=dict(
        title='% change',
        titlefont_size=13,
        tickfont_size=12,
        ))

fig_growth2.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="right",
    x=0.99))
fig_growth2.show()

In [29]:
indicators_ptf = go.Figure()
indicators_ptf.layout.template = CHART_THEME
indicators_ptf.add_trace(go.Indicator(
    mode = "number+delta",
    value = kpi_portfolio7d_pct,
    number = {'suffix': " %"},
    title = {"text": "<br><span style='font-size:0.7em;color:gray'>7 Days</span>"},
    delta = {'position': "bottom", 'reference': kpi_sp500_7d_pct, 'relative': False},
    domain = {'row': 0, 'column': 0}))

indicators_ptf.add_trace(go.Indicator(
    mode = "number+delta",
    value = kpi_portfolio15d_pct,
    number = {'suffix': " %"},
    title = {"text": "<span style='font-size:0.7em;color:gray'>15 Days</span>"},
    delta = {'position': "bottom", 'reference': kpi_sp500_15d_pct, 'relative': False},
    domain = {'row': 1, 'column': 0}))

indicators_ptf.add_trace(go.Indicator(
    mode = "number+delta",
    value = kpi_portfolio30d_pct,
    number = {'suffix': " %"},
    title = {"text": "<span style='font-size:0.7em;color:gray'>30 Days</span>"},
    delta = {'position': "bottom", 'reference': kpi_sp500_30d_pct, 'relative': False},
    domain = {'row': 2, 'column': 0}))

indicators_ptf.add_trace(go.Indicator(
    mode = "number+delta",
    value = kpi_portfolio200d_pct,
    number = {'suffix': " %"},
    title = {"text": "<span style='font-size:0.7em;color:gray'>200 Days</span>"},
    delta = {'position': "bottom", 'reference': kpi_sp500_200d_pct, 'relative': False},
    domain = {'row': 3, 'column': 1}))

indicators_ptf.update_layout(
    grid = {'rows': 4, 'columns': 1, 'pattern': "independent"},
    margin=dict(l=50, r=50, t=30, b=30)
)

In [30]:
indicators_sp500 = go.Figure()
indicators_sp500.layout.template = CHART_THEME
indicators_sp500.add_trace(go.Indicator(
    mode = "number+delta",
    value = kpi_sp500_7d_pct,
    number = {'suffix': " %"},
    title = {"text": "<br><span style='font-size:0.7em;color:gray'>7 Days</span>"},
    domain = {'row': 0, 'column': 0}))

indicators_sp500.add_trace(go.Indicator(
    mode = "number+delta",
    value = kpi_sp500_15d_pct,
    number = {'suffix': " %"},
    title = {"text": "<span style='font-size:0.7em;color:gray'>15 Days</span>"},
    domain = {'row': 1, 'column': 0}))

indicators_sp500.add_trace(go.Indicator(
    mode = "number+delta",
    value = kpi_sp500_30d_pct,
    number = {'suffix': " %"},
    title = {"text": "<span style='font-size:0.7em;color:gray'>30 Days</span>"},
    domain = {'row': 2, 'column': 0}))

indicators_sp500.add_trace(go.Indicator(
    mode = "number+delta",
    value = kpi_sp500_200d_pct,
    number = {'suffix': " %"},
    title = {"text": "<span style='font-size:0.7em;color:gray'>200 Days</span>"},
    domain = {'row': 3, 'column': 1}))

indicators_sp500.update_layout(
    grid = {'rows': 4, 'columns': 1, 'pattern': "independent"},
    margin=dict(l=50, r=50, t=30, b=30)
)

## Getting the prices for the TOP Stocks

In [31]:
last_positions = final_filtered.groupby(['ticker']).agg({'cml_units': 'last', 'cml_cost': 'last',
                                                'gain_loss': 'sum', 'cashflow': 'sum'}).reset_index()

In [32]:
%%time
curr_prices = []
for tick in last_positions['ticker']:
    stonk = yf.Ticker(tick)
    price = stonk.info['regularMarketPrice']
    curr_prices.append(price)
    print(f'Done for {tick}')
len(curr_prices)

Done for AAPL
Done for AMD
Done for AMZN
Done for BA
Done for CVX
Done for DIS
Done for DKNG
Done for FB
Done for FVRR
Done for GILT
Done for HD
Done for IBM
Done for LULU
Done for LUNA
Done for MCD
Done for MGM
Done for MMM
Done for MO
Done for MRNA
Done for NIO
Done for NVDA
Done for NWS
Done for PLL
Done for PLTR
Done for PLUG
Done for PTON
Done for QCOM
Done for RUN
Done for SHOP
Done for SYF
Done for TLRY
Done for TSLA
Done for TTD
Done for TWTR
Done for VIAC
Done for WMT
Done for WYNN
Done for XLNX
Done for ZM
Wall time: 2min 8s


39

In [33]:
last_positions['price'] = curr_prices
last_positions['current_value'] = (last_positions.price * last_positions.cml_units).round(2)
last_positions['avg_price'] = (last_positions.cml_cost / last_positions.cml_units).round(2)
last_positions = last_positions.sort_values(by='current_value', ascending=False)

In [34]:
last_positions

ticker  cml_units   cml_cost  gain_loss  cashflow      price  \
8    FVRR    14.2500   867.8250   183.2550   -684.57   160.7700   
2    AMZN     0.6800  1687.6300     0.0000  -1687.63  3185.5600   
0    AAPL    15.0000  1139.9500     0.0000  -1139.95   126.2500   
3      BA     7.4000  1279.6100     0.0000  -1279.61   225.9900   
25   PTON    12.5000   574.5643    24.3243   -550.24    88.5000   
34   VIAC    25.5000   400.6050   218.7850   -181.82    39.5700   
31   TSLA     1.5000   387.1660   453.4560     66.29   602.4900   
22    PLL    12.8000   472.7700     0.0000   -472.77    61.0528   
11    IBM     5.0000   621.4000     0.0000   -621.40   145.1100   
20   NVDA     1.0500   309.2300     0.0000   -309.23   560.3800   
4     CVX     5.4000   482.2000     0.0000   -482.20   107.8000   
18   MRNA     3.7500   251.8500     0.0000   -251.85   150.8500   
17     MO     9.0000   355.4400     0.0000   -355.44    50.3650   
15    MGM    12.0000   277.4400     0.0000   -277.44    37.7700   
32    TTD     0.9000   274.1900     0.0000   -274.19   503.0000   
35    WMT     3.0000   372.7000     0.0000   -372.70   138.8700   
27    RUN    10.0000   336.3862   537.4662    201.08    40.8100   
10     HD     1.2300   310.8000     0.0000   -310.80   326.2900   
28   SHOP     0.3600   290.2800     9.7000   -280.58  1082.9800   
14    MCD     1.6500   322.6500     0.0000   -322.65   230.7100   
13   LUNA    33.0000   210.2100     0.0000   -210.21    11.2900   
33   TWTR     6.7500   225.1500   161.5700    -63.58    51.8700   
6    DKNG     8.2500   324.0900     0.0000   -324.09    41.5300   
19    NIO     9.0000   254.0340   230.9040    -23.13    34.3800   
12   LULU     0.9000   245.8100     0.0000   -245.81   309.4200   
29    SYF     6.0000   139.7700     0.0000   -139.77    45.4000   
21    NWS     9.0000    81.7200   206.2500    124.53    25.4200   
16    MMM     1.0000   180.9500     0.0000   -180.95   204.0000   
7      FB     0.6500   117.4000     0.0000   -117.40   309.5400   
23   PLTR     9.6000   253.9000     0.0000   -253.90    20.0000   
5     DIS     0.9200    98.4000     0.0000    -98.40   172.9300   
1     AMD     2.0000   108.0000     0.0000   -108.00    74.0200   
9    GILT    15.9000   246.4170   -23.2030   -269.62     8.6400   
36   WYNN     1.0000   105.9500     0.0000   -105.95   123.6250   
24   PLUG     5.0000    27.5500   511.9000    484.35    22.7750   
37   XLNX     0.7773    90.5400     0.0000    -90.54   118.2900   
26   QCOM     0.7065    90.5400     0.0000    -90.54   126.0285   
30   TLRY     0.0000     0.0000   -25.3700    -25.37    15.3700   
38     ZM     0.0000     0.0000   377.2800    377.28   291.0500   

    current_value  avg_price  
8         2290.97      60.90  
2         2166.18    2481.81  
0         1893.75      76.00  
3         1672.33     172.92  
25        1106.25      45.97  
34        1009.04      15.71  
31         903.74     258.11  
22         781.48      36.94  
11         725.55     124.28  
20         588.40     294.50  
4          582.12      89.30  
18         565.69      67.16  
17         453.28      39.49  
15         453.24      23.12  
32         452.70     304.66  
35         416.61     124.23  
27         408.10      33.64  
10         401.34     252.68  
28         389.87     806.33  
14         380.67     195.55  
13         372.57       6.37  
33         350.12      33.36  
6          342.62      39.28  
19         309.42      28.23  
12         278.48     273.12  
29         272.40      23.30  
21         228.78       9.08  
16         204.00     180.95  
7          201.20     180.62  
23         192.00      26.45  
5          159.10     106.96  
1          148.04      54.00  
9          137.38      15.50  
36         123.62     105.95  
24         113.88       5.51  
37          91.95     116.48  
26          89.04     128.15  
30           0.00        NaN  
38           0.00        NaN

In [35]:
donut_top = go.Figure()
donut_top.layout.template = CHART_THEME
donut_top.add_trace(go.Pie(labels=last_positions.head(15).ticker, values=last_positions.head(15).current_value))
donut_top.update_traces(hole=.7, hoverinfo="label+value+percent")
donut_top.update_traces(textposition='outside', textinfo='label+value')
donut_top.update_layout(showlegend=False)
donut_top.update_layout(margin = dict(t=50, b=50, l=25, r=25))
donut_top.show()

In [36]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.FLATLY])

app.layout = dbc.Container(
    [
        dbc.Row(dbc.Col(html.H2('PORTFOLIO OVERVIEW', className='text-center text-primary, mb-3'))),  # header row
        
        dbc.Row([  # start of second row
            dbc.Col([  # first column on second row
            html.H5('Total Portfolio Value ($USD)', className='text-center'),
            dcc.Graph(id='chrt-portfolio-main',
                      figure=chart_ptfvalue,
                      style={'height':550}),
            html.Hr(),
            ], width={'size': 8, 'offset': 0, 'order': 1}),  # width first column on second row
            dbc.Col([  # second column on second row
            html.H5('Portfolio', className='text-center'),
            dcc.Graph(id='indicators-ptf',
                      figure=indicators_ptf,
                      style={'height':550}),
            html.Hr()
            ], width={'size': 2, 'offset': 0, 'order': 2}),  # width second column on second row
            dbc.Col([  # third column on second row
            html.H5('S&P500', className='text-center'),
            dcc.Graph(id='indicators-sp',
                      figure=indicators_sp500,
                      style={'height':550}),
            html.Hr()
            ], width={'size': 2, 'offset': 0, 'order': 3}),  # width third column on second row
        ]),  # end of second row
        
        dbc.Row([  # start of third row
            dbc.Col([  # first column on third row
                html.H5('Monthly Return (%)', className='text-center'),
                dcc.Graph(id='chrt-portfolio-secondary',
                      figure=fig_growth2,
                      style={'height':380}),
            ], width={'size': 8, 'offset': 0, 'order': 1}),  # width first column on second row
            dbc.Col([  # second column on third row
                html.H5('Top 15 Holdings', className='text-center'),
                dcc.Graph(id='pie-top15',
                      figure = donut_top,
                      style={'height':380}),
            ], width={'size': 4, 'offset': 0, 'order': 2}),  # width second column on second row
        ])  # end of third row
        
    ], fluid=True)

if __name__ == "__main__":
    app.run_server(debug=True, port=8058)

Dash app running on http://127.0.0.1:8058/


In [37]:
app._terminate_server_for_port("localhost",8058)

In [94]:
SIDEBAR_STYLE = {
    'position': 'fixed',
    'top': 0,
    'left': 0,
    'bottom': 0,
    'width': '12rem',
    'padding': '2rem 1rem',
    'background-color': 'lightgray',
}
CONTENT_STYLE = {
    'margin-left': '15rem',
    'margin-right': '2rem',
    'padding': '2rem' '1rem',
}

child = dbc.Container(
    [
        dbc.Row(dbc.Col(html.H2('PORTFOLIO OVERVIEW', className='text-center text-primary, mb-3'))),
        dbc.Row([
            dbc.Col([
            html.H5('Total Portfolio Value ($USD)', className='text-center'),
            dcc.Graph(id='chrt-portfolio-main',
                      figure=chart_ptfvalue,
                      style={'height':550}),
            html.Hr(),

            ],
                width={'size': 8, 'offset': 0, 'order': 1}),
            dbc.Col([
            html.H5('Portfolio', className='text-center'),
            dcc.Graph(id='indicators-ptf',
                      figure=indicators_ptf,
                      style={'height':550}),
            html.Hr()
            ],
                width={'size': 2, 'offset': 0, 'order': 2}),
            dbc.Col([
            html.H5('S&P500', className='text-center'),
            dcc.Graph(id='indicators-sp',
                      figure=indicators_sp500,
                      style={'height':550}),
            html.Hr()
            ],
                width={'size': 2, 'offset': 0, 'order': 3}),
        ]),  # end of second row
        dbc.Row([
            dbc.Col([
                html.H5('Monthly Return (%)', className='text-center'),
                dcc.Graph(id='chrt-portfolio-secondary',
                      figure=fig_growth2,
                      style={'height':380}),
            ],
                width={'size': 8, 'offset': 0, 'order': 1}),
            dbc.Col([
                html.H5('Top 15 Holdings', className='text-center'),
                dcc.Graph(id='pie-top15',
                      figure = donut_top,
                      style={'height':380}),
            ],
                width={'size': 4, 'offset': 0, 'order': 2}),
        ])
        
    ], fluid=True)

sidebar = html.Div(
    [
#         html.H5("Navigation Menu", className='display-6'),
        html.Hr(),
        html.P('Navigation Menu', className='text-center'),
        
        dbc.Nav(
            [
                dbc.NavLink('Home', href="/", active='exact'),
                dbc.NavLink('Page2', href="/page-2", active='exact')
            ],
            vertical=True,
            pills=True,
        ),
    ],
    style=SIDEBAR_STYLE,
)

content = html.Div(id='page-content', children=child, style=CONTENT_STYLE)

app = JupyterDash(__name__, external_stylesheets=[dbc.themes.SPACELAB])
# app = JupyterDash(__name__)
app.layout = html.Div([
    dcc.Location(id='url'),
    sidebar,
    content
])

if __name__ == "__main__":
    app.run_server(debug=True, port=8056)

Dash app running on http://127.0.0.1:8056/
